# Creation


In [374]:
documentDir = "test"

### 1. Tokenization

In [375]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TweetTokenizer
from collections import defaultdict, Counter
from string import punctuation
import os

tokenizer = TweetTokenizer()

def tokenize_document(content):
    
    sentences = sent_tokenize(content)
    tokens = []
    for _sent in sentences:
        sent_tokens = tokenizer.tokenize(_sent)
        sent_tokens = [_tok.lower() for _tok in sent_tokens if _tok not in punctuation]
        tokens += sent_tokens
    
    return tokens

### 2. Linguistic modules


Here there are no linguistic modules yet
Possible are:
- stemmer
- lemmatizer 

In [376]:
def preprocess_document(content):
    return tokenize_document(content)

In [377]:
def prepare_dataset(documents_dir):

    tokenized_documents = []
    for document in os.listdir(documents_dir):
        with open(os.path.join(documents_dir, document), errors='ignore',encoding='utf8') as outf:
            tokenized_documents.append(preprocess_document(outf.read()))
    print("Found documents: ", len(tokenized_documents))
    
    return tokenized_documents 

In [378]:

tokenized_documents = prepare_dataset(documentDir)
print(tokenized_documents)

Found documents:  3
[['one', 'one'], ['two', 'one', 'two', 'two'], ['two', 'tree']]


### 3. Indexer

In [379]:
from os import scandir # can be used for easier iteration of documents in a folder
# can check is_file() on the objects returned by scan_dir 
# contain whole document path, so no need to join with the directory

def get_token_doc_id_pairs(category_dir):

    token_docid = []
    doc_ids = {}

    for i, document in enumerate(scandir(category_dir)):
        if document.is_file():
            doc_ids[i] = document.name
            with open(document,encoding='utf8') as out_fp:
                document_tokens = preprocess_document(out_fp.read())
                token_docid += [(token, i) for token in document_tokens]
    return token_docid, doc_ids

In [380]:
token_docid, doc_ids = get_token_doc_id_pairs(documentDir)
print(doc_ids)
token_docid

{0: 'one.txt', 1: 'one_two.txt', 2: 'two_tree.txt'}


[('one', 0),
 ('one', 0),
 ('two', 1),
 ('one', 1),
 ('two', 1),
 ('two', 1),
 ('two', 2),
 ('tree', 2)]

Sort by tokens to form the dictionary?

In [381]:
from operator import itemgetter
sorted_token_docid = sorted(token_docid, key=itemgetter(0))
sorted_token_docid

[('one', 0),
 ('one', 0),
 ('one', 1),
 ('tree', 2),
 ('two', 1),
 ('two', 1),
 ('two', 1),
 ('two', 2)]

In [382]:
def sqash_token_in_doc(sorted_token_docid):
    """
    Returns a list of (token, doc_id, term_freq) tuples from a sorted list of (token, doc_id) list, 
    where if a token appears n times in a doc_id, we merge it in a tuple (toke, doc_id, n).
    """
    merged_tokens_in_doc = []
    for token, doc_id in sorted_token_docid:
        if merged_tokens_in_doc:
            prev_tok, prev_doc_id, prev_freq = merged_tokens_in_doc[-1]
            if prev_tok == token and prev_doc_id == doc_id:     
                merged_tokens_in_doc[-1] = (token, doc_id, prev_freq+1)
            else:
                merged_tokens_in_doc.append((token, doc_id, 1))
        else:
            merged_tokens_in_doc.append((token, doc_id, 1))
    return merged_tokens_in_doc

In [383]:
merged_tokens_in_doc = sqash_token_in_doc(sorted_token_docid)
merged_tokens_in_doc

[('one', 0, 2), ('one', 1, 1), ('tree', 2, 1), ('two', 1, 3), ('two', 2, 1)]

In [384]:
from collections import defaultdict
dictionary = defaultdict(lambda: (0, 0)) # term : doc_freq, tot freq
postings = defaultdict(lambda: []) # term: doc_ids, doc_freq

for token, doc_id, doc_freq in merged_tokens_in_doc:
    dictionary[token] = (dictionary[token][0]+1, dictionary[token][1
    ]+doc_freq)

# usually implemented as linked lists
for token, doc_id, doc_freq in merged_tokens_in_doc:
    postings[token].append((doc_id, doc_freq)) 

In [385]:
doc_ids

{0: 'one.txt', 1: 'one_two.txt', 2: 'two_tree.txt'}

In [386]:
dictionary["one"],dictionary['two'],dictionary['tree'],dictionary['zero']

((2, 3), (2, 4), (1, 1), (0, 0))

In [387]:
postings["one"],postings['two'],postings['tree'],postings['zero']

([(0, 2), (1, 1)], [(1, 3), (2, 1)], [(2, 1)], [])

# Search

### 1. Opearations functions

In [388]:
def and_query(postings_word1, postings_word2):
 
    documents_results = []
    
    postings_ind1, postings_ind2 = 0, 0
    while postings_ind1 < len(postings_word1) and postings_ind2 < len(postings_word2):
        doc_id1, doc_id2 = postings_word1[postings_ind1][0], postings_word2[postings_ind2][0]
        if doc_id1 == doc_id2:
            documents_results.append((doc_id1,0))
            postings_ind1 += 1
            postings_ind2 += 1
        elif doc_id1 < doc_id2:
            postings_ind1 += 1
        elif doc_id1 > doc_id2:
            postings_ind2 += 1
    return documents_results

In [389]:
def or_query(postings_word1, postings_word2):

    documents_results = []
    
    postings_ind1, postings_ind2 = 0, 0
    while postings_ind1 < len(postings_word1) and postings_ind2 < len(postings_word2):
        doc_id1, doc_id2 = postings_word1[postings_ind1][0], postings_word2[postings_ind2][0]
        if doc_id1 == doc_id2:
            documents_results.append((doc_id1,0))
            postings_ind1 += 1
            postings_ind2 += 1
        elif doc_id1 < doc_id2:
            documents_results.append((doc_id1,0))
            postings_ind1 += 1
        elif doc_id1 > doc_id2:
            documents_results.append((doc_id2,0))
            postings_ind2 += 1
    if postings_ind1 == len(postings_word1):
        for i in range(postings_ind2,len(postings_word2)):
            documents_results.append((postings_word2[i][0],0))
    if postings_ind2 == len(postings_word2):
        for i in range(postings_ind1,len(postings_word1)):
            documents_results.append((postings_word1[i][0],0))
    return documents_results

In [390]:
def not_query(postings_word):
    document_count = len(tokenized_documents)
    documents_results = []

    prev = 0
    for i in range(0,len(postings_word)):
        for not_doc in range(prev,postings_word[i][0]):
            documents_results.append((not_doc,0))
        prev = postings_word[i][0]+1
    for not_doc in range(prev,document_count):
        documents_results.append((not_doc,0))
    
    return documents_results
        

In [391]:
print(not_query(postings["one"]))
print(not_query(postings["two"]))
print(not_query(postings["tree"]))

[(2, 0)]
[(0, 0)]
[(0, 0), (1, 0)]


### 2. Parsing classes

In [392]:

from typing import Callable, Iterable

class BoolRetrievalOperand:
    def __init__(self, t):
        self.label = t[0]
        print("Creating BoolRetrievalOperand" + str(t))
    
    def process(self) -> list:
        print("Processing BoolRetrievalOperand "+ self.label)
        self.value = postings[self.label]
        print(self.value)
        return self.value

    def __str__(self) -> str:
        return self.label

    __repr__ = __str__


In [393]:
class BoolRetrievalNot:
    def __init__(self, t):
        print("Creating BoolRetrievalNot"+str(t))
        self.arg = t[0][1]
        print(self.arg)


    def process(self) -> list:
        res = not_query(self.arg.process())
        print("Processing "+str(self))
        print(res)
        return res

    def __str__(self) -> str:
        return "~" + str(self.arg)

    __repr__ = __str__


In [394]:
class BoolRetrievalBinOp:
    repr_symbol: str = ""
    eval_fn: Callable[
        [Iterable[list]], list
    ] = lambda _: []

    def __init__(self, t):
        print("Creating BoolRetrievalBinOp "+self.repr_symbol)
        print(t)
        self.args = t[0][0::2]

    def __str__(self) -> str:
        sep = " %s " % self.repr_symbol
        return "(" + sep.join(map(str, self.args)) + ")"


    def process(self) -> list:
        res = self.eval_fn(a.process() for a in self.args)
        print("Processing "+str(self))
        print(res)
        return res

    __repr__ = __str__

def and_helper(a,bb) -> list:
    b = []
    for i in bb:
        b.append(i)
    prev = b[0]
    for i in range(0,len(b)-1):
        prev = and_query(prev,b[i+1])
    return prev

def or_helper(a,bb) -> list:
    b = []
    for i in bb:
        b.append(i)
    prev = b[0]
    for i in range(0,len(b)-1):
        print(prev)
        prev = or_query(prev,b[i+1])
    return prev

class BoolRetrievalAnd(BoolRetrievalBinOp):
    repr_symbol = "&"
    eval_fn = and_helper


class BoolRetrievalOr(BoolRetrievalBinOp):
    repr_symbol = "|"
    eval_fn = or_helper


### 3. Parsing grammar

In [395]:

from pyparsing import infixNotation, opAssoc, Keyword, Word, alphas, ParserElement,pyparsing_unicode, nums

NOT = Keyword("not")
AND = Keyword("and")
OR = Keyword("or")
token = Word(pyparsing_unicode.printables)
token.setParseAction(BoolRetrievalOperand).setName("token")

boolOperand = token
boolOperand.setName("bool_operand")

# define expression, based on expression operand and
# list of operations in precedence order
boolExpr = infixNotation(
    boolOperand,
    [
        (NOT, 1, opAssoc.RIGHT,BoolRetrievalNot),
        (AND, 2, opAssoc.LEFT,BoolRetrievalAnd),
        (OR, 2, opAssoc.LEFT,BoolRetrievalOr),
    ],
).setName("boolean_expression")


### 4. Testing

In [396]:
tests = [
    ("not not one",True),
    ("one", True),
    ("tree", True),
    ("one and tree",True),
    ("one and two",True),
    ("two and two",True),
    ("two and (two and one)",True),
    ("one and tree and two",True),
    ("one or tree",True),
    ("(one or tree) and two",True),
    ("not((one or tree) and two)",True),
]


for test_string, expected in tests:
    res = boolExpr.parseString(test_string)[0]
    success = "test"#"PASS" if bool(res) == expected else "FAIL"
    print("Query: "+test_string, "\n", res, "=", str(res.process()), "\n", success, "\n")

Creating BoolRetrievalOperand['one']
Creating BoolRetrievalNot[['not', one]]
one
Creating BoolRetrievalNot[['not', ~one]]
~one
Processing BoolRetrievalOperand one
[(0, 2), (1, 1)]
Processing ~one
[(2, 0)]
Processing ~~one
[(0, 0), (1, 0)]
Query: not not one 
 ~~one = [(0, 0), (1, 0)] 
 test 

Creating BoolRetrievalOperand['one']
Processing BoolRetrievalOperand one
[(0, 2), (1, 1)]
Query: one 
 one = [(0, 2), (1, 1)] 
 test 

Creating BoolRetrievalOperand['tree']
Processing BoolRetrievalOperand tree
[(2, 1)]
Query: tree 
 tree = [(2, 1)] 
 test 

Creating BoolRetrievalOperand['one']
Creating BoolRetrievalOperand['tree']
Creating BoolRetrievalBinOp &
[[one, 'and', tree]]
Processing BoolRetrievalOperand one
[(0, 2), (1, 1)]
Processing BoolRetrievalOperand tree
[(2, 1)]
Processing (one & tree)
[]
Query: one and tree 
 (one & tree) = [] 
 test 

Creating BoolRetrievalOperand['one']
Creating BoolRetrievalOperand['two']
Creating BoolRetrievalBinOp &
[[one, 'and', two]]
Processing BoolRetrieva

# Real test

In [397]:
documentDir="people"
tokenized_documents = prepare_dataset(documentDir)
# print(tokenized_documents)
token_docid, doc_ids = get_token_doc_id_pairs(documentDir)
sorted_token_docid = sorted(token_docid, key=itemgetter(0))
merged_tokens_in_doc = sqash_token_in_doc(sorted_token_docid)

dictionary = defaultdict(lambda: (0, 0)) # term : doc_freq, tot freq
postings = defaultdict(lambda: []) # term: doc_ids, doc_freq

print(doc_ids)
for token, doc_id, doc_freq in merged_tokens_in_doc:
    dictionary[token] = (dictionary[token][0]+1, dictionary[token][1
    ]+doc_freq)

# usually implemented as linked lists
for token, doc_id, doc_freq in merged_tokens_in_doc:
    postings[token].append((doc_id, doc_freq)) 

query = "inventor and Renaissance"
res = boolExpr.parseString(query)[0]
print(res.process())

query = "inventor and renaissance"
res = boolExpr.parseString(query)[0]
res.process()

Found documents:  3
{0: 'galileo_galilei.txt', 1: 'isaac_newton.txt', 2: 'leonardo_da_vinci.txt'}
Creating BoolRetrievalOperand['inventor']
Creating BoolRetrievalOperand['Renaissance']
Creating BoolRetrievalBinOp &
[[inventor, 'and', Renaissance]]
Processing BoolRetrievalOperand inventor
[(2, 1)]
Processing BoolRetrievalOperand Renaissance
[]
Processing (inventor & Renaissance)
[]
[]
Creating BoolRetrievalOperand['inventor']
Creating BoolRetrievalOperand['renaissance']
Creating BoolRetrievalBinOp &
[[inventor, 'and', renaissance]]
Processing BoolRetrievalOperand inventor
[(2, 1)]
Processing BoolRetrievalOperand renaissance
[(0, 1), (2, 8)]
Processing (inventor & renaissance)
[(2, 0)]


[(2, 0)]

In [398]:
documentDir=r"C:\Users\Victor\OneDrive\Documents\Second brain\2. Knowledge base\Finals\Themes"
tokenized_documents = prepare_dataset(documentDir)
# print(tokenized_documents)
token_docid, doc_ids = get_token_doc_id_pairs(documentDir)
sorted_token_docid = sorted(token_docid, key=itemgetter(0))
merged_tokens_in_doc = sqash_token_in_doc(sorted_token_docid)

dictionary = defaultdict(lambda: (0, 0)) # term : doc_freq, tot freq
postings = defaultdict(lambda: []) # term: doc_ids, doc_freq

print(doc_ids)
for token, doc_id, doc_freq in merged_tokens_in_doc:
    dictionary[token] = (dictionary[token][0]+1, dictionary[token][1
    ]+doc_freq)

# usually implemented as linked lists
for token, doc_id, doc_freq in merged_tokens_in_doc:
    postings[token].append((doc_id, doc_freq)) 

query = "algorithm or algorithms"
res = boolExpr.parseString(query)[0]
print(res.process())

query = "алгоритъм or алгоритми"
res = boolExpr.parseString(query)[0]
res.process()

Found documents:  27
{0: '1. Графи.md', 1: '10. Нормални форми на БД.md', 2: '11. C++ basics.md', 3: '12. ООП.md', 4: '13. Структури от данни и алгоритми.md', 5: '14. Софтуерно инженерство.md', 6: '15. Разпределени системи.md', 7: '16. GUI.md', 8: '17. QA.md', 9: '18. Софтуерни архитектури.md', 10: '19. Requirenments engineering.md', 11: '2. Булеви функции.md', 12: '20. ПИСС.md', 13: '21. Project management.md', 14: '22. XML.md', 15: '23. Taylor series.md', 16: '24. Integrals.md', 17: '25. Vector space and algebra.md', 18: '26. Polynomials.md', 19: '27. Statistics.md', 20: '3. Автомати.md', 21: '4. Граматики.md', 22: '5. CPU.md', 23: '6. Memory pages and segments, interrupts.md', 24: '7. Processes in OS.md', 25: '8. Networking.md', 26: '9. Релационен модел.md'}
Creating BoolRetrievalOperand['algorithm']
Creating BoolRetrievalOperand['algorithms']
Creating BoolRetrievalBinOp |
[[algorithm, 'or', algorithms]]
Processing BoolRetrievalOperand algorithm
[]
Processing BoolRetrievalOperand al

[(1, 0), (4, 0), (17, 0), (18, 0), (20, 0)]

In [399]:
greet = Word(alphas) + "," + Word(alphas) + "!"
for greeting_str in [
            "Hello, World!",
            "Bonjour, Monde!",
            "Hola, Mundo!",
            "Hallo, Welt!",
        ]:
    greeting = greet.parse_string(greeting_str)
    print(greeting)

['Hello', ',', 'World', '!']
['Bonjour', ',', 'Monde', '!']
['Hola', ',', 'Mundo', '!']
['Hallo', ',', 'Welt', '!']


In [401]:

class BoolRetrievalWildcard:
    def __init__(self, t):
        self.label = t[0]
        print("Creating BoolRetrievalWildcard" + str(t))
    
    def process(self) -> list:
        print("Processing BoolRetrievalWildcard " + str(self))
        self.value = postings[ self.label]
        print(self.value)
        return [self.value]

    def __str__(self) -> str:
        return self.label+"*"

    __repr__ = __str__

class BoolRetrievalProximity:
    def __init__(self, t):
        self.A = t[0]
        self.positions = t[2]
        self.B = t[3]
        print("Creating BoolRetrievalProximity" + str(t))
    
    def process(self) -> list:
        print("Processing BoolRetrievalProximity " + self.A + " \\" + self.positions + " " + self.B)
       # self.value = postings[ self.label]
      #  print(self.value)
        return []

    def __str__(self) -> str:
        return "" + self.A + " \\" + self.positions + " " + self.B

    __repr__ = __str__

class BoolRetrievalSentence:
    def __init__(self, t):
        self.words = t[1:]
        print("Creating BoolRetrievalSentence" + str(t))
    
    def process(self) -> list:
        print("Processing BoolRetrievalSentence " + str(self.words))
       # self.value = postings[ self.label]
      #  print(self.value)
        return []

    def __str__(self) -> str:
        return "/s"+str(self.words)

    __repr__ = __str__


token = Word(alphas)
token.setParseAction(BoolRetrievalOperand).setName("token")

wildcard =  Word(alphas) + "*"
wildcard.setParseAction(BoolRetrievalWildcard).setName("wildcard")

#   Word(alphas) + ("/" + Word(nums) + Word(alphas))[1,...]
proximity = Word(alphas) + "/" + Word(nums) + Word(alphas)
proximity.setParseAction(BoolRetrievalProximity).setName("proximity")

sentence = "/s" + Word(alphas)[1,...]
sentence.setParseAction(BoolRetrievalSentence).setName("proximity")

boolOperand = sentence | proximity | wildcard | token

# define expression, based on expression operand and
# list of operations in precedence order
boolExpr = infixNotation(
    boolOperand,
    [
        (NOT, 1, opAssoc.RIGHT,BoolRetrievalNot),
        (AND, 2, opAssoc.LEFT,BoolRetrievalAnd),
        (OR, 2, opAssoc.LEFT,BoolRetrievalOr),
    ],
)

tests = [
    ("asdasda*",True),
    ("one", True),
    ("tree", True),
    ("one /4 tree", True),
    ("one /4 tree /5 five", True),
    ("/s one tree five six", True),
   # ("one or tree*",True),
    ("(one* and (/s what this)) or (will /3 be)",True),
]


for test_string, expected in tests:
    res = boolExpr.parseString(test_string)[0]
    success = "test"#"PASS" if bool(res) == expected else "FAIL"
    print("Query: "+test_string, "\n", res, "=", str(res.process()), "\n", success, "\n")


Creating BoolRetrievalWildcard['asdasda', '*']
Processing BoolRetrievalWildcard asdasda*
[]
Query: asdasda* 
 asdasda* = [[]] 
 test 

Creating BoolRetrievalOperand['one']
Processing BoolRetrievalOperand one
[]
Query: one 
 one = [] 
 test 

Creating BoolRetrievalOperand['tree']
Processing BoolRetrievalOperand tree
[]
Query: tree 
 tree = [] 
 test 

Creating BoolRetrievalProximity['one', '/', '4', 'tree']
Processing BoolRetrievalProximity one \4 tree
Query: one /4 tree 
 TODO = [] 
 test 

Creating BoolRetrievalProximity['one', '/', '4', 'tree']
Processing BoolRetrievalProximity one \4 tree
Query: one /4 tree /5 five 
 TODO = [] 
 test 

Creating BoolRetrievalSentence['/s', 'one', 'tree', 'five', 'six']
Processing BoolRetrievalSentence ['one', 'tree', 'five', 'six']
Query: /s one tree five six 
 TODO = [] 
 test 

Creating BoolRetrievalWildcard['one', '*']
Creating BoolRetrievalSentence['/s', 'what', 'this']
Creating BoolRetrievalBinOp &
[[one*, 'and', TODO]]
Creating BoolRetrievalPro